In [2]:
import pandas as pd
from tdc.utils import get_label_map

In [28]:
pd.set_option('display.max_rows', None)

In [3]:
twosides = pd.read_csv('../raw_data/twosides.csv')

In [13]:
twosides_labels = get_label_map(name = 'TWOSIDES', task = 'DDI', name_column = 'Side Effect Name')

In [23]:
twosides['side_effect'] = twosides['Y'].apply(lambda x: twosides_labels[x])

In [29]:
twosides['side_effect'].value_counts().tail(100)

mast cell disease                 93
alcoholic cirrhosis               92
brachial plexus injury            89
gastric stasis                    89
peripheral nerve injury           89
varicocele                        89
hypercalcinuria                   87
retinopathy of prematurity        86
Dermatitis diaper                 86
cavernous sinus thrombosis        86
encopresis                        81
familial hypercholesterolemia     81
acute glomerulonephritis          80
Polio                             80
salivary gland enlargement        79
thyroid adenoma                   77
neuroblastoma                     76
pneumonectomy                     73
parapsoriasis                     72
cryptorchidism                    72
hypervitaminosis                  72
acute intermittent porphyria      71
oral neoplasm                     70
adrenal carcinoma                 68
xanthoma                          67
prolactinoma                      66
erythremia                        66
r

In [195]:
# convert labels into dataframe
twosides_labels_df = pd.DataFrame(twosides_labels.values(), index=twosides_labels.keys(), columns=['side_effect'])

In [196]:
# cleaning data by removing uppercase
twosides_labels_df['side_effect'] = twosides_labels_df['side_effect'].apply(lambda x: x.lower())

In [197]:
twosides_labels_df = twosides_labels_df.sort_index()

In [202]:
twosides_labels_df.to_excel('twosides_labels.xlsx')

In [198]:
twosides_labels_df['meddra'] = 0

In [199]:
twosides_labels_df

,side_effect,meddra
0,abdominal distension,0
1,abdominal pain,0
2,birth defect,0
3,abortion spontaneous,0
4,abortion missed,0
...,...,...
1312,dysphonia,0
1313,morphea,0
1314,eosinophilic pneumonia,0
1315,retinal vein thrombosis,0


In [192]:
reclassification_df = pd.read_excel('../raw_data/sub_sys.xlsx')

In [193]:
reclassification_df = reclassification_df.replace(np.nan, 'zzz')

In [194]:
reclassification_df

,bone,joints & collagen(connective tissues),teeth,muscles+tendon,heart,blood,blood vessels+arteries,spleen+bone marrow+lymph nodes+tonsils+misc_immune,mouth+salivary gland+throat,oesophagus+stomach+small intestine+large intestine+gastrointestinal trac,...,tongue,nose,lungs+bronchi+misc_respiratory,kidneys+urinary bladder+gall bladder+prostate gland+urethra+misc_excretory,genitalia+vagina+uterus+ovaries+misc_reproductive_female,genitalia+testis & penis+misc_reproductive_male,breast,skin+subcutaneous fat,hair,nails
0,Ankle fracture,Arthralgia,Alveolar osteitis,Asthenia,Acute coronary syndrome,Abnormal clotting factor,Angiopathy,Hypersplenism,Burning mouth syndrome,Aphagia,...,Ageusia,Acute sinusitis,Acute interstitial pneumonitis,Acute phosphate nephropathy,Genital burning sensation,Genital burning sensation,Benign breast neoplasm,Abscess,Alopecia,Ingrowing nail
1,Atypical femur fracture,Arthritis,Bruxism,Asthenic conditions,Acute myocardial infarction,Acetonaemia,Application site haematoma,Spleen disorder,Dry mouth,Barrett's oesophagus,...,Atrophic glossitis,Allergic sinusitis,Acute pulmonary oedema,Acute prerenal failure,Genital candidiasis,Genital candidiasis,Breast abscess,Abscess limb,Androgenetic alopecia,Nail bed infection
2,Atypical fracture,Arthritis infective,Dental caries,Blood magnesium decreased,Adams-Stokes syndrome,Acquired haemophilia,Arteriosclerosis,Spleen palpable,Foot and mouth disease,Erosive oesophagitis,...,Dysgeusia,Anosmia,Acute respiratory distress syndrome,Albumin urine present,Genital discharge,Genital discharge,Breast atrophy,Abscess sterile,Dandruff,Nail bed tenderness
3,Bone and joint infections,Arthropathy,Dental discomfort,Blood magnesium increased,Angina pectoris,Activated partial thromboplastin time prolonged,Arteriovenous fistula,Splenic infarction,Mouth haemorrhage,Gastrooesophageal reflux disease,...,Epiglottitis,Bradyarrhythmia,Acute respiratory failure,Albuminuria,Genital discomfort,Genital discomfort,Breast calcifications,Acanthosis nigricans,Diffuse alopecia,Nail discolouration
4,Bone cancer metastatic,Chondrolysis,Dentofacial anomaly,Cataplexy,Angina unstable,Activated partial thromboplastin time shortened,Bleeding time prolonged,Splenic peliosis,Mouth injury,Gastrooesophageal sphincter insufficiency,...,Glossitis,Chronic sinusitis,Alveolar proteinosis,Aminoaciduria,Genital disorder female,Genital disorder female,Breast cancer,Acarodermatitis,Furuncle,Nail discomfort
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,zzz,zzz,zzz,zzz,zzz,Calcium intoxication,zzz,zzz,zzz,zzz,...,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz
378,zzz,zzz,zzz,zzz,zzz,Diabetic hyperglycaemic coma,zzz,zzz,zzz,zzz,...,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz
379,zzz,zzz,zzz,zzz,zzz,Diabetic hyperosmolar coma,zzz,zzz,zzz,zzz,...,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz
380,zzz,zzz,zzz,zzz,zzz,Diabetic ketoacidosis,zzz,zzz,zzz,zzz,...,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz,zzz


In [200]:
for organ in reclassification_df.columns:
    for side_effect in reclassification_df[organ]:
        twosides_labels_df['meddra'][twosides_labels_df['side_effect'] == side_effect.lower()] = organ.strip()

In [201]:
len(twosides_labels_df['meddra'].unique())

31

In [65]:
# creating a new column labelled meddra
twosides_labels_df['meddra'] = 0

# Using CTCAE to help with reclassification of targets

In [105]:
ctcae = pd.read_excel('../raw_data/CTCAE_v5.0.xlsx')
ctcae = ctcae[['MedDRA SOC', 'CTCAE Term']]
ctcae['CTCAE Term'] = ctcae['CTCAE Term'].apply(lambda x: x.lower())

In [115]:
ctcae['Zipped'] = list(zip(ctcae['MedDRA SOC'], ctcae['CTCAE Term']))

In [119]:
for meddra_term, side_effect in ctcae['Zipped']:
    twosides_labels_df['meddra'][twosides_labels_df['side_effect'].str.contains(side_effect)] = meddra_term

In [122]:
twosides_labels_df['meddra'].value_counts()

0                                                       1014
Gastrointestinal disorders                                39
Musculoskeletal and connective tissue disorders           37
Nervous system disorders                                  30
Infections and infestations                               26
Respiratory, thoracic and mediastinal disorders           20
General disorders and administration site conditions      20
Injury, poisoning and procedural complications            20
Vascular disorders                                        19
Cardiac disorders                                         16
Eye disorders                                             15
Metabolism and nutrition disorders                        14
Skin and subcutaneous tissue disorders                     9
Renal and urinary disorders                                8
Reproductive system and breast disorders                   8
Psychiatric disorders                                      6
Hepatobiliary disorders 

# Using lists of common words to reclassify targets

In [139]:
meddra_se = {
    'Blood and lymphatic system disorders': ['anemia', 'artery', 'vein'],
    'Cardiac disorders': ['heart', 'cardiac'],
    'Congenital, familial and genetic disorders': [],
    'Ear and labyrinth disorders': [],
    'Endocrine disorders': [],
    'Eye disorders': ['eye'],
    'Gastrointestinal disorders': ['gastric'],
    'General disorders and administration site conditions': [],
    'Hepatobiliary disorders': ['liver', 'hepa'],
    'Immune system disorders': [],
    'Infections and infestations': ['infection'],
    'Injury, poisoning and procedural complications': [],
    'Investigations': [],
    'Metabolism and nutrition disorders': [],
    'Musculoskeletal and connective tissue disorders': ['fracture', 'muscle', 'bone', 'joint'],
    'Neoplasms benign, malignant and unspecified (incl cysts and polyps)': ['cancer', 'carcinoma', 'sarcoma', 'blastoma'],
    'Nervous system disorders': ['spinal'],
    'Pregnancy, puerperium and perinatal conditions': ['preg'],
    'Product issues': [],
    'Psychiatric disorders': [],
    'Renal and urinary disorders': ['kidney', 'urin'],
    'Reproductive system and breast disorders': ['menstrual'],
    'Respiratory, thoracic and mediastinal disorders': [],
    'Skin and subcutaneous tissue disorders': ['skin'],
    'Social circumstances': [],
    'Surgical and medical procedures': [],
    'Vascular disorders': []
}

In [ ]:
two

In [150]:
list(twosides_labels_df['side_effect'][twosides_labels_df['meddra'] == 0])

['back ache',
 'lung edema',
 'agitated',
 'abnormal movements',
 'peliosis',
 'rupture of spleen',
 'drug hypersensitivity',
 'allergies',
 'thrombocytopenia',
 'bradycardia',
 'lung infiltration',
 'bleeding',
 'hypoglycaemia neonatal',
 'gastrointestinal obstruction',
 'hyperglycaemia',
 'peritonitis',
 'hypoglycaemia',
 'cerebral infarct',
 'difficulty breathing',
 'decreased body temperature',
 'hypomagnesaemia',
 'neonatal respiratory distress syndrome',
 'acute respiratory distress syndrome',
 'lipoma',
 'appendectomy',
 'strabismus',
 'blood calcium decreased',
 'hypercapnia',
 'intestinal perforation',
 'leucocytosis',
 'disorder lung',
 'hot flash',
 'arterial pressure nos decreased',
 'bladder inflammation',
 'patent ductus arteriosis',
 'arterial pressure nos increased',
 'hypoventilation',
 'edema',
 'aspartate aminotransferase increase',
 'cold extremities',
 'anaphylactic reaction',
 'eruption',
 'neumonia',
 'adenocarcinoma',
 'malnourished',
 'haemorrhoids',
 'still bi

In [163]:
twosides_labels_df[twosides_labels_df['side_effect'].str.contains('breast')]

,side_effect,meddra
1260,breast cancer,0
546,breast pain,Reproductive system and breast disorders
110,breast cyst,0
526,breast lump,0
322,breast dysplasia,0
452,breast enlargement,0
111,breast disorder,0
1162,breast tenderness,0
1000,breast discharge,0
112,breast swelling,0


In [140]:
for key, value in meddra_se.items():
    for keyword in value:
        twosides_labels_df['meddra'][twosides_labels_df['side_effect'].str.contains(keyword)] = key

In [143]:
twosides_labels_df['meddra'].value_counts()

0                                                       954
Musculoskeletal and connective tissue disorders          49
Gastrointestinal disorders                               44
Infections and infestations                              41
Nervous system disorders                                 33
Cardiac disorders                                        29
Eye disorders                                            21
Respiratory, thoracic and mediastinal disorders          20
General disorders and administration site conditions     18
Vascular disorders                                       18
Skin and subcutaneous tissue disorders                   18
Renal and urinary disorders                              15
Metabolism and nutrition disorders                       14
Hepatobiliary disorders                                  10
Reproductive system and breast disorders                  8
Psychiatric disorders                                     6
Injury, poisoning and procedural complic

In [66]:
twosides_labels_df['meddra'][twosides_labels_df['side_effect'].str.contains('bone')] = 'Musculoskeletal and connective tissue disorders'

In [69]:
twosides_labels_df['meddra'][twosides_labels_df['side_effect'].str.contains('muscle')] = 'Musculoskeletal and connective tissue disorders'

In [ ]:
= 'Musculoskeletal and connective tissue disorders'